# Tratamento e Normalização - Conversões T2

## Configuração e Importações

### Objetivo: Esse pipeline tem por objetivo normalizar os labels de pesquisa que estejam divergentes entre versões de pesquisa criados.

In [3]:
# Importações básicas
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import os
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Adiciona src ao path
sys.path.append('../src')

# Utilitários de dados
from data_utils import (
    load_raw_data,
    save_processed_data,
    remove_duplicates,
    handle_missing_values,
    detect_outliers,
    normalize_column,
    process_phone_number,
    clean_and_lower_column
)

# Utilitários SQL
from sql_utils import DatabaseConnection as Dbc, load_query_from_file

# Utilitários de visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Utilitários de API
from api_utils import (
    make_request,
    get_json,
    post_json,
    paginated_request,
    response_to_dataframe
)

# Configurações pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Load Database Driver
db = Dbc()

print('✓ Importações concluídas com sucesso!')

✓ Importações concluídas com sucesso!


# Buscar Conversões T2 no banco


In [4]:
# Buscar Conversões T2

query = load_query_from_file('select_conversion_t2')
df_t2 = db.execute_query(query)

## Definindo protocolos de Substituição


In [6]:
labels_to_replace = {
    "biggest_fluency_challange": "biggest_fluency_challenge",
    "english_proficiency_level": "english_level",
    "monthly_income": "monthly_income",
    "monthly_incomme": "monthly_income",
    "monthly_personal_income": "monthly_income",
    "monthy_income": "monthly_income",
    "has_taken_english_course": "took_english_course",
    "took_english_course_before": "took_english_course",
}

## Iterando por raw_infos e corrigindo Labels

In [7]:
# Antes de iterar, crie as colunas necessárias no DataFrame
df_t2['new_conversion_raw_info'] = None
df_t2['modified_labels'] = False

# Agora, faça a iteração normalmente
for idx, row in df_t2.iterrows():
    data = row.get("conversion_raw_info")
    new_data = {}
    modified = False

    if isinstance(data, dict):  # Garante que data é um dicionário antes de iterar
        for key, value in data.items():
            if key in labels_to_replace:
                modified = True
                new_data[labels_to_replace[key]] = value
            else:
                new_data[key] = value
    else:
        # Se data não for um dicionário, apenas atribua o valor como está
        new_data = data

    df_t2.at[idx, 'new_conversion_raw_info'] = new_data
    df_t2.at[idx, 'modified_labels'] = modified


# Checking Modifications

In [8]:
import pandas as pd

def sample_comparacao_labels(df, coluna_modificada='modified_labels', coluna_antiga='conversion_raw_info', coluna_nova='new_conversion_raw_info', n_sample=100):
    """
    Seleciona um sample de linhas modificadas e compara os labels alterados.

    Args:
        df (pd.DataFrame): DataFrame com os dados.
        coluna_modificada (str): Nome da coluna booleana que indica modificação.
        coluna_antiga (str): Nome da coluna com os labels antigos.
        coluna_nova (str): Nome da coluna com os labels novos.
        n_sample (int): Número de exemplos no sample.

    Returns:
        diffs (list): Lista de dicionários com as diferenças detectadas.
    """
    modified_true = df[df[coluna_modificada] == True]
    amostra = modified_true.sample(min(len(modified_true), n_sample), random_state=42)
    diffs = []

    for idx, row in amostra.iterrows():
        old = row.get(coluna_antiga)
        new = row.get(coluna_nova)
        if isinstance(old, dict) and isinstance(new, dict):
            old_keys = set(old.keys())
            new_keys = set(new.keys())
            diferencas = {
                "idx": idx,
                "antigos_sem_novos": list(old_keys - new_keys),
                "novos_sem_antigos": list(new_keys - old_keys),
                "valores_atualizados": {k: {"antigo": old[k], "novo": new[k]} for k in old_keys & new_keys if old[k] != new[k]}
            }
            diffs.append({
                "idx": idx,
                "chaves_antigas": list(old.keys()),
                "chaves_novas": list(new.keys()),
                "diferencas": diferencas,
                "old": old,
                "new": new
            })
        else:
            diffs.append({
                "idx": idx,
                "old": old,
                "new": new,
                "diferencas": "Um dos valores não é dicionário."
            })

    print(f"Sample de {len(diffs)} objetos modificados com comparação de labels (mostrando até 3 exemplos):\n")
    for d in diffs[:3]:  # Mostra só os 3 primeiros exemplos para consulta rápida
        print(f"Idx: {d['idx']}")
        print("Chaves antigas:", d.get("chaves_antigas"))
        print("Chaves novas :", d.get("chaves_novas"))
        print("Diferenças   :", d["diferencas"])
        print("-" * 30)
    return diffs

# Exemplo de uso:
diffs_sample = sample_comparacao_labels(df_t2)


Sample de 100 objetos modificados com comparação de labels (mostrando até 3 exemplos):

Idx: 80264
Chaves antigas: ['form_id', 'token', 'campaign_id', 'score', 'country', 'gender', 'age_range', 'current_occupation', 'biggest_fluency_desire', 'monthy_income', 'english_level', 'took_english_course']
Chaves novas : ['form_id', 'token', 'campaign_id', 'score', 'country', 'gender', 'age_range', 'current_occupation', 'biggest_fluency_desire', 'monthly_income', 'english_level', 'took_english_course']
Diferenças   : {'idx': 80264, 'antigos_sem_novos': ['monthy_income'], 'novos_sem_antigos': ['monthly_income'], 'valores_atualizados': {}}
------------------------------
Idx: 57701
Chaves antigas: ['score', 'token', 'gender', 'country', 'form_id', 'age_range', 'age_score', 'teb_score', 'reprocessed', 'gender_score', 'has_interest', 'income_score', 'english_level', 'monthy_income', 'occupation_score', 'current_occupation', 'took_english_course', 'biggest_fluency_desire']
Chaves novas : ['score', 't

In [ ]:
# ✅ NOVO: 10.000 registros = ~5 segundos (36x mais rápido!)
update_query = load_query_from_file('update_conversion_t2_raw_info')

resultado = db.update_from_dataframe(
    df=df_t2,
    query=update_query,
    column_mapping={
        "id": "conversion_id",
        "new_conversion_raw_info": "new_conversion_raw_info"
    },
    filter_column="new_conversion_raw_info",  # Filtra apenas não-nulos
    batch_size=1000,
    show_progress=True
)

print(f"✅ Sucesso: {resultado['success']}")
print(f"❌ Falhas: {resultado['failed']}")
print(f"⏱️ Tempo: {resultado['elapsed_time']:.2f}s")

Filtrando por coluna 'new_conversion_raw_info': 195862/195862 registros
[12:00:15] Iniciando batch update...
Total de registros: 195862
Tamanho do lote: 1000
[12:04:59] Lote 1/196 - Processados: 1000/195862 (0.5%) - Linhas afetadas neste lote: 1000
[12:07:45] Lote 2/196 - Processados: 2000/195862 (1.0%) - Linhas afetadas neste lote: 1000
[12:10:15] Lote 3/196 - Processados: 3000/195862 (1.5%) - Linhas afetadas neste lote: 1000
[12:12:44] Lote 4/196 - Processados: 4000/195862 (2.0%) - Linhas afetadas neste lote: 1000
[12:15:14] Lote 5/196 - Processados: 5000/195862 (2.6%) - Linhas afetadas neste lote: 1000
[12:17:43] Lote 6/196 - Processados: 6000/195862 (3.1%) - Linhas afetadas neste lote: 1000
[12:20:12] Lote 7/196 - Processados: 7000/195862 (3.6%) - Linhas afetadas neste lote: 1000
[12:22:40] Lote 8/196 - Processados: 8000/195862 (4.1%) - Linhas afetadas neste lote: 1000
[12:25:12] Lote 9/196 - Processados: 9000/195862 (4.6%) - Linhas afetadas neste lote: 1000
[12:27:40] Lote 10/196 